# Run ground truth examples and get expected results


## Setup

### Notebook Pre-Req

You can upload this notebook into a Jupyter environment configured to use Neptune Workbench. I tested this on an Amazon Sagemaker notebook running Python 3.10.x. 

### Python Pre-Req

I tested this on an Amazon Sagemaker notebook running Python 3.10.x. You need Python 3.9 or higher.

### Neptune Pre-Req

Your Neptune cluster must run engine version 1.2.x or higher

## Open ground truth file and run the queries against Neptune


In [1]:
!mkdir -p expected

In [5]:
import yaml
import boto3
import utilities as u
from pathlib import Path

# get ground truth and prefixes
resources = Path.cwd() / "resources"
pfx = (Path.cwd() / "resources" / "prefixes.txt").read_text()
ground_truth = yaml.safe_load((resources / "ground-truth.yaml").read_text())
session=boto3.Session()

for index, q in enumerate(ground_truth):
    print(str(index))
    nlq=q['question']
    sparql=f"""
        {pfx}
        
        {q['SPARQL']}
        
        LIMIT 20
    """

    try:
        res=u.execute_sparql(sparql, session) 
        u.write_sparql_res("expected", str(index), nlq, q['SPARQL'], sparql, res, "")
    except Exception as e:
        print(f"Error on {index}")
        print("Exception: {}".format(type(e).__name__))
        print("Exception message: {}".format(e))
        error_msg="Exception message: {}".format(e).replace("\n", " ")
        u.write_sparql_res("expected", str(index), nlq, q['SPARQL'], sparql, {}, error_msg)
        print(error_msg)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
